In [ ]:
from pathlib import Path
import pandas as pd

file_path = Path("data/marked_sentences_472_cutted.csv")

try:

    slavery = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        slavery = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        slavery = pd.read_csv(file_path, encoding='utf-8', header=None)

slavery

Или:

In [ ]:
from pathlib import Path
import pandas as pd

file_path = Path("data/unmarked_sentences_cutted.csv")

try:

    all_cases = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        all_cases = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        all_cases = pd.read_csv(file_path, encoding='utf-8', header=None)

all_cases

In [ ]:
import pandas as pd
import google.generativeai as genai
import json
import time
import math


genai.configure(api_key="ВАШ_АПИ_КЛЮЧ")

# Функция для оценки количества токенов
def estimate_tokens(text):
    return math.ceil(len(text) / 3)  # Оценка: 1 токен ≈ 3 символа

# Функция для обработки батча
def classify_batch(batch):
    try:
        prompt = """ТЕКСТ ИНСТРУКЦИИ
        Проанализируй следующие приговоры и верни JSON в формате:
        [
            {{"case_number": "<номер дела>", "slavery_detected": true/false, "reason": "<обоснование>", "victim": <количество жертв (если есть, иначе 0)>}},
            ...
        ]

        **Приговоры**:
        """

        for case_number, text in batch:
            prompt += f"\n\nНомер дела: {case_number}\nТекст: {text}\n---"

        model = genai.GenerativeModel("gemini-2.5-pro-exp-03-25")
        response = model.generate_content(prompt)

        response_text = response.text.strip()
        print(f"Ответ модели:\n{response_text}")

        return safe_json_parse(response_text)

    except Exception as e:
        print(f"Ошибка обработки batch {batch}: {e}")
        return [{"case_number": case_number, "slavery_detected": None, "reason": "Ошибка обработки", "victim": 0} for case_number, _ in batch]

# Функция для парсинга JSON
def safe_json_parse(response_text):
    try:
        response_text = response_text.strip("```json").strip("```").strip()
        return json.loads(response_text)
    except json.JSONDecodeError:
        print(f"Ошибка декодирования JSON: {response_text}")
        return []

if "slavery" not in locals():
    raise ValueError("slavery не определен!")

slavery = slavery.drop_duplicates()

if "text" not in slavery.columns:
    raise ValueError("Файл не содержит колонку 'text'!")

# Оцениваем количество токенов в каждом приговоре
slavery["num_tokens"] = slavery["text"].apply(estimate_tokens)

# Разбиваем приговоры на батчи по ≤ 200K токенов
batch = []
current_tokens = 0
batches = []

for _, row in slavery.iterrows():
    case_number, text, tokens = row["case_number"], row["text"], row["num_tokens"]
    
    if current_tokens + tokens > 195000:
        batches.append(batch)
        batch = []
        current_tokens = 0
    
    batch.append((case_number, text))
    current_tokens += tokens

if batch:
    batches.append(batch)

results = []
for batch in batches:
    batch_results = classify_batch(batch)
    results.extend(batch_results)

    print("Ожидание 10 секунд перед следующим запросом...")
    time.sleep(10)

results_df = pd.DataFrame(results)
results_df.to_csv("тест1.csv", index=False)

print("Классификация завершена! Результат сохранен в 'тест1.csv'")


In [ ]:
import pandas as pd

file_path = Path("data/gemini25_version3.csv")

try:

    slavery_classified = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        slavery_classified = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        slavery_classified = pd.read_csv(file_path, encoding='utf-8', header=None)

slavery_classified.rename(columns={'text_x': 'text'}, inplace=True)



In [ ]:
#Выполнение этой ячейки необходимо, если результат классификации ИИ не был предварительно соединен с размеченным датафреймом
df_merged = slavery.merge(slavery_classified, on=['case_number'], how='left', indicator=True)

In [ ]:
value_counts = df_merged['slavery_detected'].value_counts()
print(value_counts)

non_na_rows = df_merged[df_merged['slavery_detected'].notna()]
len(non_na_rows)


In [ ]:
df_merged = df_merged.dropna(subset=['slavery_detected'])
df_merged['slavery_detected'] = df_merged['slavery_detected'].map({True: 1, False: 0})
#slavery['slavery_detected'] = slavery['slavery_detected'].fillna(0).astype(int)


In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns


y_true = df_merged['Slavery_1_step']  # Ручная разметка
y_pred = df_merged['slavery_detected']  # Разметка ИИ

# Рассчитываем ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Выводим Classification Report для классификации 1
print("\nClassification Report - slav_1:")
print(classification_report(y_true, y_pred))

# Рассчитываем взвешенную F1-меру вручную
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\nWeighted F1 Score: {weighted_f1:.4f}")

# Рассчитываем macro-average F1-меру вручную
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\nMacro-average F1 Score: {macro_f1:.4f}")

# Выводим матрицу ошибок для slav_1
cm_1 = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix - slav_2:")
print(cm_1)

# Визуализируем матрицу ошибок
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

sns.heatmap(cm_1, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'], ax=axes[0])
axes[0].set_title('Confusion Matrix - slav_2')
axes[0].set_xlabel('Predicted')
axes
